**Thông tin nhóm: Nhóm 2**

**20120083 - Nguyễn Trọng Hiếu**

**20120105 - Lê Hoàng Huy**


### 1. Application description
**What is your chosen application?**

Our chosen application: Thực hiện và tối ưu hóa forward-pass của convolutional layer bằng CUDA.

<li> Fashion MNIST </li>

* Input

* Output

* Use cases

<li> convolutional layer</li>

* Input

* Output

* Use cases

**Does it need to speed up? Why**


### 2. Sequential implementation
**Design: Describe steps to go from input to output (don’t show code)**

Dựa vào mô hình base <a href='https://github.com/iamhankai/mini-dnn-cpp'>mini-dnn-cpp</a> tiến hành 
* Xây dựng kiến trúc <a href='https://www.kaggle.com/code/blurredmachine/lenet-architecture-a-complete-guide'>LeNet-5</a>
* Chạy trên bộ dữ liệu <a href='https://github.com/zalandoresearch/fashion-mnist'>Fashion MNIST</a>

**Evaluate:**
* Describe your experiment setup
* Runthe code to see results
* Doesit run correctly?

### 3. Parallel implementation
**Analyze: Which steps do you parallelize? Why these steps?**

Thực hiện song song trên forward-pass của convolutional layer.
Nhận xét tính khả thi trong việc tận dụng tính toán song song của GPU để tiết kiệm thời gian.

**Design: How do you parallelize? (don’t show code)**

**Evaluate:**

* Describe your experiment setup
* Runthe code to see results
* Doesit run correctly & faster? If not, do you know why?

### 4. Parallel implementation + optimization

**Version 1**

**Analyze: Which parts (often: which kernels) do you optimize? Why these parts?**

**Design: How do you optimize? (don’t show code)**

**Evaluate:**

* Describe your experiment setup
* Runthe code to see results
* Doesit run correctly & faster? If not, do you know why?

**Version 2**

**Analyze: Which parts (often: which kernels) do you optimize? Why these parts?**

**Design: How do you optimize? (don’t show code)**

**Evaluate:**

* Describe your experiment setup
* Runthe code to see results
* Doesit run correctly & faster? If not, do you know why?

### 5. Reflection
* Eachmember: What difficulties have you encountered?
* Eachmember: What have you learned?
* Your team: If you had more time, what would you do?